In [1]:
import pandas as pd
import numpy as np
import os
import json
from fix_addresses_master import *

In [2]:
#Read in the address junction table 
df = pd.read_csv('~/Dropbox (GaTech)/CDS-2019-AlbanyHub/ToDatabase/addr_junct_table.csv')
directory = '/home/mirabel/Dropbox (GaTech)/CDS-2019-AlbanyHub/Processed-Data/attom_json' #dir of json files

In [3]:
#read all json files into a single list
j_list=[]
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        f = open(directory+'/'+filename)
        j_list.append(json.load(f))
        f.close()
    

In [4]:
#Look at all the values available for a single home
example_entry = j_list[0][0]
example_entry.keys()

dict_keys(['identifier', 'lot', 'area', 'address', 'location', 'summary', 'utilities', 'sale', 'building', 'assessment', 'vintage'])

In [5]:
#concatenate j_list into a single list
data = [y for x in j_list for y in x]
len(data)

30721

In [6]:
def prop_split(data):
    #separate these out by the major keys
    all_identifiers = [x['identifier'] for x in data if len(x)>0]
    all_lot = [x['lot'] for x in data if len(x)>0]
    all_area = [x['area'] for x in data if len(x)>0]
    all_addresses = [x['address'] for x in data if len(x)>0]
    all_location = [x['location'] for x in data if len(x)>0]
    all_summary = [x['summary'] for x in data if len(x)>0]
    all_utilities = [x['utilities']for x in data if len(x)>0]
    all_sale = [x['sale'] for x in data if len(x)>0]
    all_building = [x['building'] for x in data if len(x)>0]
    all_assessment = [x['assessment'] for x in data if len(x)>0]
    all_vintage = [x['vintage'] for x in data if len(x)>0]
    #convert data from a multi-layered dict into a single dict for a pandas dataframe
    dict_full = {
        'address':[x['line1'] for x in all_addresses],
        'lot_size':[x['lotSize1'] for x in all_lot],
        'zoningType': [x.get('zoningType') for x in all_lot],
        'siteZoningIdent': [x.get('siteZoningIdent') for x in all_lot],
        'propClass': [x['propClass'] for x in all_summary],
        'yearBuilt':[x['yearBuilt'] for x in all_summary],
        'size':[x['size']['grossSizeAdjusted'] for x in all_building],
        'baths':[x['rooms']['bathsTotal'] for x in all_building],
        'beds':[x['rooms']['beds'] for x in all_building],
        'rooms':[x['rooms']['roomsTotal'] for x in all_building],
        'floors':[x['interior'].get('floors') for x in all_building],
        'condition':[x['construction'].get('condition') for x in all_building],
        'foundationType':[x['construction'].get('foundationType') for x in all_building],
        'roofCover':[x['construction'].get('roofCover') for x in all_building],
        'wallType':[x['construction'].get('wallType') for x in all_building],
        'improvementYear':[x['construction'].get('propertyStructureMajorImprovementsYear') for x in all_building],
        'assessment':[x['assessed']['assdTtlValue'] for x in all_assessment],
        'market':[x['market']['mktTtlValue'] for x in all_assessment]
    }
    return pd.DataFrame(data=dict_full)

In [7]:
#Convert to pandas dataframe
dfp = prop_split(data)
dfp.drop_duplicates(subset="address")
dfp.index = range(len(dfp))
len(dfp)

26931

In [8]:
#get the primary ids in the junction table for each address returned
id_dict = {df.loc[x,'Address']:df.loc[x, 'Id'] for x in range(len(df))} #maps address to id, from junct table
NOT_FOUND = id_dict['NOT FOUND']
prim_ids=[id_dict.get(x, NOT_FOUND) for x in fix_series(dfp['address'])] #id of each addresss in new dataframe


In [9]:
#find the locs where x is not in the address junction table
nf = [dfp.loc[x, 'address'] for x in range(len(prim_ids)) if prim_ids[x]==NOT_FOUND]

In [10]:
len(nf)

659

In [11]:
len(nf)/len(prim_ids) #about 3% of addresses were not found despite the successful api call

0.0244699417028703

In [12]:
dfp['PrimaryId'] = prim_ids

In [13]:
#replace 0's with None for clarity
dfp.loc[dfp['yearBuilt'] == 0, 'yearBuilt']=None
dfp.loc[dfp['size']==0, 'size'] = None
dfp.loc[dfp['lot_size'] == 0, 'lot_size'] = None
dfp.loc[dfp['assessment'] ==0, 'assessment']=None
dfp.loc[dfp['market'] == 0, 'market']=None
#Drop rows where address is Not found
dfp.loc[dfp['PrimaryId'] == NOT_FOUND, :] = None
dfp = dfp.dropna(how='all')


### Look at all the missing data and try to load it again

In [14]:
missing = df[~df['Id'].isin(dfp['PrimaryId'])]
#missing.to_csv("missing.csv", index=False)

In [ ]:
dfp.to_csv(directory+'/property_data.csv', index=False)

## Data Exploration
### 1. Identifiers
Address (Primary ID) - linked with junction table

### 2. Counts of Observations
Get counts and percentages of data

In [ ]:
len(dfp)/len(data) # about 86% of addresses have data

In [ ]:
len(dfp[dfp['size'].notnull()])/len(data) #about 83% of addresses have info on square footage

In [ ]:
len(dfp[dfp['yearBuilt'].notnull()])/len(data) #similarly about 83% of addresses have year built

In [ ]:
len(data) #number of addresses in the database

In [ ]:
len(dfp) #number of addresses successfully retrieved from database

### 3. Some example records

In [ ]:
dfp.loc[[5000, 10000, 15000, 20000, 25000], :]

### 4. Summarization

In [ ]:
#all fields
dfp.columns

In [ ]:
def get_stats(l):
    print("Min:", min(l))
    print("Max:", max(l))
    print("Mean:",l.mean())

In [ ]:
print("Lot Size")
get_stats(dfp['lot_size'])
print("Year Built")
get_stats(dfp['yearBuilt'])
print("Size (sq ft)")
get_stats(dfp['size'])
print('assessment')
get_stats(dfp['assessment'])
print('market')
get_stats(dfp['market'])

In [ ]:
# tiny lot size
dfp.loc[dfp['lot_size']<0.02, :]

In [ ]:
# large lot size
dfp.loc[dfp['lot_size']>10000, :] # this is an error

In [ ]:
# Small size
dfp[dfp['size']<250] # nothing ridiculous

In [ ]:
#large size
dfp[dfp['size']>200000] #largest are commercial, industrial, distribution,etc - checks out

#### Question: difference between market and assessment? (market is often >2x assessment value)
https://www.realtor.com/advice/sell/assessed-value-vs-market-value-difference/<br>
Market: what home could sell for<br>
Assessed: Used for property tax<br>

In [ ]:
dfp[dfp['market']<500] #most low values are vacant properties, 5200 radium springs is a power plant

In [ ]:
dfp[dfp['market']>10000000]

In [ ]:
dfp[dfp['assessment']<500] #Vacant properties

#### Categorical values

In [ ]:
dfp['zoningType'].value_counts()

In [ ]:
dfp['siteZoningIdent'].value_counts()

In [ ]:
dfp['propClass'].value_counts()

In [ ]:
dfp['floors'].value_counts()

In [ ]:
dfp['wallType'].value_counts()

In [ ]:
dfp['condition'].value_counts()

In [ ]:
dfp['roofCover'].value_counts()

In [ ]:
dfp['foundationType'].value_counts()

### 5. Statistical summaries by groups

In [ ]:
dfp_g1 = dfp.loc[:,['zoningType','lot_size', 'yearBuilt', 'size', 'market', 'assessment']].groupby(by='zoningType')
pd.set_option('precision', 2)
dfp_g1.agg(['mean', 'min', 'max'])

In [ ]:
dfp_g2 = dfp.loc[:, ['propClass','lot_size', 'yearBuilt', 'size', 'market', 'assessment']].groupby(by='propClass')
dfp_g2.agg(['mean', 'min', 'max'])

In [ ]:
dfp_g2 = dfp.loc[:, ['condition','lot_size', 'yearBuilt', 'size', 'market', 'assessment']].groupby(by='condition')
dfp_g2.agg(['mean', 'min', 'max'])

In [ ]:
dfp_g3 = dfp.groupby(by=['condition', 'propClass']) 
dfp_g3.count()

In [ ]:
dfp_g4 = dfp.groupby(by=['siteZoningIdent', 'propClass']) 
dfp_g4.count()

# Read missing back in

In [17]:
missing.index = range(len(missing))
missing.shape

(4583, 7)

In [18]:
missing_dir = '/home/mirabel/Dropbox (GaTech)/CDS-2019-AlbanyHub/Processed-Data/attom_json/missing'
#read all json files into a single list
j_list=[]
flist = os.listdir(missing_dir)
flist.sort()
for filename in flist:
    if filename.endswith('.json'):
        f = open(missing_dir+'/'+filename)
        j_list.append(json.load(f))
        f.close()

In [19]:
data_new = [y for x in j_list for y in x]
data_found =[]#The data returned by the attom api
orig_addresses = [] #The addresses used to query the attom api
missing_list = pd.DataFrame(data={'Address':[], 'StatusMSG':[], 'Code':[]}) #the return values of addresses which were still not coded
for i in range(len(data_new)):
    if len(data_new[i])==8 or len(data_new[i])==6:
        missing_list = missing_list.append({'Address':missing.iloc[i, 1], 'StatusMSG':data_new[i]['msg'], 'Code':data_new[i]['code']}, ignore_index=True)
    elif len(data_new[i])==0:
        missing_list = missing_list.append({'Address':missing.iloc[i, 1], 'StatusMSG':'unknown', 'Code':0}, ignore_index=True)
    else:
        data_found.append(data_new[i])
        orig_addresses.append(missing.iloc[i, 1])

In [21]:
len(data_found)

793

In [92]:
#Projects that were found when the api was queried again
dfp2 = prop_split(data_found)
dfp2.drop_duplicates(subset="address")
dfp2.index = range(len(dfp2))
#get the primary ids in the junction table for each address returned
id_dict = {df.loc[x,'Address']:df.loc[x, 'Id'] for x in range(len(df))} #maps address to id, from junct table
NOT_FOUND = id_dict['NOT FOUND']
prim_ids=[id_dict.get(x, NOT_FOUND) for x in orig_addresses] #id of each addresss in new dataframe
dfp2['PrimaryId'] = prim_ids


In [93]:
#Compare the addresses returned by attom api to those which were queried -> is there a difference?
data_found_lat = [float(x['location']['latitude']) for x in data_found]
data_found_lon = [float(x['location']['longitude']) for x in data_found]
data_found_acc = [x['location']['accuracy'] for x in data_found]
df_subset = df[df['Address'].isin(orig_addresses)]
df_subset.index = range(len(df_subset))

df_compare = pd.DataFrame(data={'found_addresses':dfp2['address'], 'orig_addresses':df_subset['Address'], 'accuracy':data_found_acc,'found_lat':data_found_lat, 'found_lon':data_found_lon, 'orig_lat':df_subset['Xcoord'], 'orig_lon':df_subset['Ycoord']})
df_compare['coord_diff'] = df_compare['found_lat']+df_compare['found_lon']-df_compare['orig_lat']-df_compare['orig_lon']

In [95]:
#Addresses which may have been miscoded: difference in coordinates is significant
df_compare[df_compare['coord_diff']>0.005]
#US highway addresses: found is more accurate
#all else: original is more accurate
#Despite incorrect geocoding, likely only the E/W N/S mixups and wildly incorrect addresses are wrong
#That is, 80, 97, 190, 553, 676, 734
drop_idx = [80, 97, 190, 553, 676, 734]
nf_miscoded = dfp2.loc[drop_idx, 'address']
dfp2.drop(drop_idx, inplace=True)
dfp2.index = range(len(dfp2))

In [96]:
dfp_all = pd.concat([dfp, dfp2], axis=0)

In [97]:
missing_list.groupby(by='StatusMSG').count()

,Address,Code
StatusMSG,,
Geocoder Results Address Not Identified.,84,84
Success without results. No data available for this address.,474,474
SuccessWithoutResult,3221,3221
System Not Responding.,11,11


In [98]:
missing_list.groupby(by='Code').count()
#-2 = System Not Responding : There is a failure within the api
#1 = SuccessWithoutResult : Your request was successful but returned no results
#210 = Geocoder Search Results Address Not Identified. : The input address could not be identified. Please try again.
#212 = 	Success without results. No data available for this address. : The input address has been located with ZIP level precision, but a record is not available.

,Address,StatusMSG
Code,,
-2.0,11,11
1.0,3221,3221
210.0,84,84
212.0,474,474


#### Rerun parts after definition of data

In [100]:
missing_list[missing_list['StatusMSG']=='System Not Responding.'].to_csv('missing_notresponding.csv')


TypeError: append() got an unexpected keyword argument 'axis'

In [58]:
missing_list[missing_list['StatusMSG']=='Success without results. No data available for this address.'].head()

,Address,StatusMSG,Code
2,235 BONNY VIEW AVE,Success without results. No data available for...,212.0
5,109 W BROAD AVE,Success without results. No data available for...,212.0
6,252 BONNY VIEW AVE,Success without results. No data available for...,212.0
7,243 BONNY VIEW AVE,Success without results. No data available for...,212.0
8,247 BONNY VIEW AVE,Success without results. No data available for...,212.0


In [55]:
missing_list[missing_list['StatusMSG']=='Geocoder Results Address Not Identified.'].head()

,Address,StatusMSG,Code
58,82 W US HIGHWAY,Geocoder Results Address Not Identified.,210.0
152,3602 BROOKHOLLOW PKWY,Geocoder Results Address Not Identified.,210.0
172,E OGLETHORPE BLVD,Geocoder Results Address Not Identified.,210.0
177,E WADDELL AVE,Geocoder Results Address Not Identified.,210.0
255,CASON ST,Geocoder Results Address Not Identified.,210.0


#### No response resolved

In [79]:
f = open('/home/mirabel/Dropbox/CDS-2019-AlbanyHub/Processed-Data/attom_json/missing/noresponse/json_dump_missing_noresponse0_10.json')
data_nr = json.load(f)
f.close()
dfp3 = prop_split(data_nr)
prim_ids=[id_dict.get(x, NOT_FOUND) for x in dfp3['address']] #id of each addresss in new dataframe
#manually fix one value
prim_ids[6] = id_dict.get('600 POLARIS DR')
dfp3['PrimaryId'] = prim_ids

In [82]:
dfp_all = pd.concat([dfp, dfp2, dfp3], axis=0)

0.8811562123628788

In [113]:
p1 = missing_list[missing_list['StatusMSG']!='System Not Responding.']
p2 = p1.append([{'Address':x} for x in nf_miscoded], ignore_index=True)
#
p2.tail(20)
p2.to_csv('missing_unresolved.csv', index=False)

80             320 N JACKSON ST
97     1212 WHISPERING PINES RD
190             200 E BROAD AVE
553            1401 E BROAD AVE
676           2100 N FULTON AVE
734                639 OWENS ST
Name: address, dtype: object